# Capstone project: Production

**Objectives/ Problem Statement**<br>
To predict the price of new HDB launch before the launch <br>
_**Input a set of features and output a price **_

**why this problem?**<br>
A practical problem that most Singaporeans (including myself) face 


In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime
import re

# import seaborn as sns
# import matplotlib.pyplot as plt
# sns.set_style('whitegrid')

# %matplotlib inline
# %config InlineBackend.figure_format = 'retina'

# Quick Cleaning 
(code and analysis taken from Part2_EDA)

In [49]:
resale[resale['year']>=2014].shape

(81347, 25)

resale.to_csv('./datasets/resale_forproduction.csv')

In [2]:
resale = pd.read_csv('./datasets/resale_forproduction.csv')
resale = resale.drop(['Unnamed: 0'],axis=1)
resale['date'] = pd.to_datetime(resale['date'])

In [3]:
resale.columns

Index(['block', 'flat_model', 'flat_type', 'floor_area_sqm',
       'lease_commence_date', 'month', 'remaining_lease', 'resale_price',
       'storey_range', 'street_name', 'town', 'latitude', 'longitude',
       'formatted_address', 'postal_code', 'date', 'year',
       'calculated_remaining_lease', 'block_num', 'flat_model_PREMIUM',
       'flat_model_IMPROVED', 'flat_model_MODEL_A', 'storey'],
      dtype='object')

In [4]:
price_time = resale.groupby(['date']).mean()[['resale_price']].reset_index()
price_time.columns = ['date','mean_resale_price']
price_time['year'] = price_time.date.dt.year

resale = resale.merge(price_time,on=['date','year'])
resale['var_resale_price'] = resale['resale_price'] - resale['mean_resale_price']

# Modelling: Regression

In [5]:
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, ElasticNet, ElasticNetCV, LassoCV, LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score

In [10]:
def Obtain_coefficients(estimator,X_train,y_train,X):
    coefficient = pd.DataFrame({'variable':X.columns,
                                'coef':estimator.coef_,
                                'abs_coef':np.abs(estimator.coef_)})
    feat_coeff = coefficient[coefficient['abs_coef']>0]
    print('{} out of {} features are selected.' .format(feat_coeff.shape[0],len(X.columns)))
    return coefficient

In [6]:
# From Part2_EDA, some of the flat models have been subsumed into 'flat_model_OTHERS' 
# Instead of dropping the first variable, we will drop 'falt_model_Others instead
# dummy_col = ['flat_type','storey','town']
dummy_col = ['storey','town']
df1 = resale[['flat_model_PREMIUM', 'flat_model_IMPROVED', 'flat_model_MODEL_A']]
df2 = pd.get_dummies(resale[['flat_model']]).drop('flat_model_OTHERS',axis = 1)
df3 = pd.get_dummies(resale[dummy_col],drop_first=True)
# df4 = resale[['latitude', 'longitude']]
df5 = resale[['lease_commence_date','floor_area_sqm']]

In [28]:
df_var['year'].unique()

array([2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2015, 2016,
       2017, 2018, 2014], dtype=int64)

In [29]:
df_var = pd.concat([df1,df2,df3,df5, resale[['var_resale_price','year','date']]],axis=1)
train_var = df[df['year']<2014]
test_var  = df[df['year']>=2014]

# X_var = df_var.drop(['var_resale_price','year','date'],axis=1)
X_var_train = train.drop(['var_resale_price','year','date'],axis=1)
y_var_train = train['var_resale_price']
X_var_test  = test.drop(['var_resale_price','year','date'],axis=1)
y_var_test  = test['var_resale_price']

In [30]:
df_var.columns

Index(['flat_model_PREMIUM', 'flat_model_IMPROVED', 'flat_model_MODEL_A',
       'flat_model_ADJOINED FLAT', 'flat_model_APARTMENT', 'flat_model_DBSS',
       'flat_model_MAISONETTE', 'flat_model_NEW GENERATION',
       'flat_model_SIMPLIFIED', 'flat_model_STANDARD', 'flat_model_TERRACE',
       'flat_model_TYPE S', 'storey', 'town_BEDOK', 'town_BISHAN',
       'town_BUKIT BATOK', 'town_BUKIT MERAH', 'town_BUKIT PANJANG',
       'town_BUKIT TIMAH', 'town_CENTRAL AREA', 'town_CHOA CHU KANG',
       'town_CLEMENTI', 'town_GEYLANG', 'town_HOUGANG', 'town_JURONG EAST',
       'town_JURONG WEST', 'town_KALLANG/WHAMPOA', 'town_MARINE PARADE',
       'town_PASIR RIS', 'town_PUNGGOL', 'town_QUEENSTOWN', 'town_SEMBAWANG',
       'town_SENGKANG', 'town_SERANGOON', 'town_TAMPINES', 'town_TOA PAYOH',
       'town_WOODLANDS', 'town_YISHUN', 'lease_commence_date',
       'floor_area_sqm', 'var_resale_price', 'year', 'date'],
      dtype='object')

In [31]:
ss=StandardScaler()
optimal_lasso = LassoCV(n_alphas=500, cv=10, verbose=1)
optimal_lasso.fit(ss.fit_transform(X_var_train),y_var_train)
lasso = Lasso(alpha=optimal_lasso.alpha_)
pipe = Pipeline(steps=[('standard scaler', ss),
                       ('lasso',lasso)])
pipe.fit(X_var_train,y_var_train)

print(pipe.score(X_var_train,y_var_train))
print(pipe.score(X_var_test,y_var_test))

#to see what features affect the model the most
coeff = Obtain_coefficients(lasso,ss.fit_transform(X_var_train),y_var_train,X_var_train)
coeff.sort_values('abs_coef',ascending=False)

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

0.8236567999858536
0.7318428852998009
37 out of 40 features are selected.


,variable,coef,abs_coef
39,floor_area_sqm,69731.605267,69731.605267
38,lease_commence_date,33375.599146,33375.599146
36,town_WOODLANDS,-28251.349601,28251.349601
25,town_JURONG WEST,-20137.384975,20137.384975
20,town_CHOA CHU KANG,-19755.784021,19755.784021
17,town_BUKIT PANJANG,-16257.490432,16257.490432
12,storey,15822.884225,15822.884225
32,town_SENGKANG,-15445.083828,15445.083828
37,town_YISHUN,-15330.679441,15330.679441
31,town_SEMBAWANG,-14676.603011,14676.603011


In [32]:
y_var_pred = pipe.predict(X_test)

In [33]:
predicted_var = pd.DataFrame({'y_pred':y_var_pred,
              'date':test['date']}).reset_index(drop=True)

# mean_price = price_time.set_index('date').to_dict()['mean_resale_price']

# prediction = [pred + mean_price[predicted.loc[i,'date']] for i,pred in enumerate(predicted['y_pred'])]

In [34]:
predicted_var.head()

,y_pred,date
0,-70917.796062,2015-09-01
1,-33309.356713,2015-09-01
2,-12304.601926,2015-09-01
3,-9553.699339,2015-09-01
4,-74572.895511,2015-09-01


In [35]:
r2_score(y_test,y_var_pred)

0.7318428852998009

# Modelling: Timeseries

In [36]:
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras import backend as K
from sklearn.preprocessing import MinMaxScaler

Using TensorFlow backend.


In [37]:
def create_data(timeseries, lag=1, as_array=True):
    if not isinstance(timeseries, pd.Series):
        timeseries = pd.Series(timeseries.ravel())
    y = timeseries[lag:]
    X = pd.DataFrame({'lag'+str(lag-i):timeseries.shift(-i) for i in range(0, lag)}).dropna().iloc[:-1, :]
    if not as_array:
        return X, y
    else:
        return X.values, y.values

In [38]:
mms = MinMaxScaler(feature_range=(-1, 1))
mean_resale_px = price_time[['mean_resale_price']]
mean_resale_px = mms.fit_transform(mean_resale_px)

In [40]:
train_mean = price_time[price_time['year']<2014]
test_mean  =  price_time[price_time['year']>=2014]

train_mean_size = len(train_mean)
test_mean_size = len(mean_resale_px) - train_mean_size

train_mean, test_mean = mean_resale_px[0:train_mean_size,:], mean_resale_px[train_mean_size:len(mean_resale_px),:]

In [41]:
lag = 1
X_mean_train, y_mean_train = create_data(train_mean, lag)
X_mean_test, y_mean_test = create_data(test_mean, lag)
X_mean_train = np.reshape(X_mean_train, (X_mean_train.shape[0], 1, X_mean_train.shape[1]))
X_mean_test= np.reshape(X_mean_test, (X_mean_test.shape[0], 1, X_mean_test.shape[1]))

In [42]:
K.clear_session()

model = Sequential()
model.add(LSTM(32, input_shape=(None, 1)))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_mean_train, y_mean_train, epochs=20, batch_size=1, verbose=1)

Epoch 1/20
107/107 [==============================] - 2s 20ms/step - loss: 0.4024
Epoch 2/20
107/107 [==============================] - 0s 3ms/step - loss: 0.1903
Epoch 3/20
107/107 [==============================] - 0s 3ms/step - loss: 0.0420
Epoch 4/20
107/107 [==============================] - 0s 3ms/step - loss: 0.0033
Epoch 5/20
107/107 [==============================] - 0s 3ms/step - loss: 0.0011
Epoch 6/20
107/107 [==============================] - 0s 3ms/step - loss: 0.0011
Epoch 7/20
107/107 [==============================] - 0s 3ms/step - loss: 0.0011
Epoch 8/20
107/107 [==============================] - 0s 3ms/step - loss: 0.0011
Epoch 9/20
107/107 [==============================] - 0s 3ms/step - loss: 0.0011
Epoch 10/20
107/107 [==============================] - 0s 3ms/step - loss: 0.0011
Epoch 11/20
107/107 [==============================] - 0s 3ms/step - loss: 0.0011
Epoch 12/20
107/107 [==============================] - 0s 3ms/step - loss: 0.0011
Epoch 13/20
107/107 [===

In [45]:
K.clear_session()

model = Sequential()
model.add(LSTM(32, input_shape=(None, 1)))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_mean_train, y_mean_train, epochs=20, batch_size=1, verbose=1)

Epoch 1/20
107/107 [==============================] - 2s 16ms/step - loss: 0.3017: 0s - loss: 0.31
Epoch 2/20
107/107 [==============================] - 0s 3ms/step - loss: 0.1052
Epoch 3/20
107/107 [==============================] - 0s 3ms/step - loss: 0.0139
Epoch 4/20
107/107 [==============================] - 0s 3ms/step - loss: 0.0015
Epoch 5/20
107/107 [==============================] - 0s 3ms/step - loss: 0.0011
Epoch 6/20
107/107 [==============================] - 0s 3ms/step - loss: 0.0011
Epoch 7/20
107/107 [==============================] - 0s 3ms/step - loss: 0.0011
Epoch 8/20
107/107 [==============================] - 0s 3ms/step - loss: 0.0011
Epoch 9/20
107/107 [==============================] - 0s 3ms/step - loss: 0.0011
Epoch 10/20
107/107 [==============================] - 0s 3ms/step - loss: 0.0011
Epoch 11/20
107/107 [==============================] - ETA: 0s - loss: 0.0013 - 0s 3ms/step - loss: 0.0012
Epoch 12/20
107/107 [==============================] - 0s 3ms/st

In [46]:
model.reset_states()
mean_train_predict = model.predict(X_mean_train)
model.reset_states()
mean_test_predict = model.predict(X_mean_test)
model.reset_states()

train_r2 = r2_score(y_mean_train, mean_train_predict)
print('train_r2',train_r2) 
test_r2 = r2_score(y_mean_test, mean_test_predict)
print('test_r2',test_r2)

train_r2 0.9978224935168247
test_r2 -0.24994329454751285


In [50]:
model.reset_states()
mean_train_predict = model.predict(X_mean_train)
model.reset_states()
mean_test_predict = model.predict(X_mean_test)
model.reset_states()

train_r2 = r2_score(y_mean_train, mean_train_predict)
print('train_r2',train_r2) 
test_r2 = r2_score(y_mean_test[:-1], mean_test_predict[1:])
print('test_r2',test_r2)

train_r2 0.9978224935168247
test_r2 0.698468288413788


In [37]:
mean_pred = price_time[price_time['year']>=2014].drop([108,160])
mean_pred['y_pred']=mms.inverse_transform(testPredict[1:])

In [38]:
mean_price = mean_pred.set_index('date').to_dict()['y_pred']

In [39]:
price_time.loc[108,'date']

Timestamp('2014-01-01 00:00:00')

In [40]:
new_predicted = predicted[(predicted['date']!= price_time.loc[108,'date']) & (predicted['date']!= price_time.loc[160,'date'])].reset_index(drop=True)

In [41]:
for i,pred in enumerate(new_predicted['y_pred']):
    print(new_predicted.loc[i,'date'])

2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00


2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00


2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00


2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00


2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00
2016-08-01 00:00:00


...

2016-01-01 00:00:00
2016-01-01 00:00:00
2016-01-01 00:00:00
2016-01-01 00:00:00
2016-01-01 00:00:00
2016-01-01 00:00:00
2016-01-01 00:00:00
2016-01-01 00:00:00
2016-01-01 00:00:00
2016-01-01 00:00:00
2016-01-01 00:00:00
2016-01-01 00:00:00
2016-01-01 00:00:00
2016-01-01 00:00:00
2016-01-01 00:00:00
2016-01-01 00:00:00
2016-01-01 00:00:00
2016-01-01 00:00:00
2016-01-01 00:00:00
2016-01-01 00:00:00
2016-01-01 00:00:00
2016-01-01 00:00:00
2016-01-01 00:00:00
2016-01-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00


2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00
2014-05-01 00:00:00


2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00


2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00
2017-04-01 00:00:00


2014-08-01 00:00:00
2014-08-01 00:00:00
2014-08-01 00:00:00
2014-08-01 00:00:00
2014-08-01 00:00:00
2014-08-01 00:00:00
2014-08-01 00:00:00
2014-08-01 00:00:00
2014-08-01 00:00:00
2014-08-01 00:00:00
2014-08-01 00:00:00
2014-08-01 00:00:00
2014-08-01 00:00:00
2014-08-01 00:00:00
2014-08-01 00:00:00
2014-08-01 00:00:00
2014-08-01 00:00:00
2014-08-01 00:00:00
2014-08-01 00:00:00
2014-08-01 00:00:00
2014-08-01 00:00:00
2014-08-01 00:00:00
2014-08-01 00:00:00
2014-08-01 00:00:00
2014-08-01 00:00:00
2014-08-01 00:00:00
2014-08-01 00:00:00
2014-08-01 00:00:00
2014-08-01 00:00:00
2014-08-01 00:00:00
2014-08-01 00:00:00
2014-08-01 00:00:00
2014-08-01 00:00:00
2014-08-01 00:00:00
2014-08-01 00:00:00
2014-08-01 00:00:00
2014-08-01 00:00:00
2014-08-01 00:00:00
2014-08-01 00:00:00
2014-08-01 00:00:00
2014-08-01 00:00:00
2014-08-01 00:00:00
2014-08-01 00:00:00
2014-08-01 00:00:00
2014-08-01 00:00:00
2014-08-01 00:00:00
2014-08-01 00:00:00
2014-08-01 00:00:00
2014-08-01 00:00:00
2014-08-01 00:00:00


In [42]:
prediction = [pred + mean_price[new_predicted.loc[i,'date']] for i,pred in enumerate(new_predicted['y_pred'])]

In [43]:
len(prediction)

78494

In [44]:
len(resale.iloc[test_index].resale_price.values)

NameError: name 'test_index' is not defined

In [ ]:
test.head()

In [ ]:
test =  resale[resale['year']>=2014]
test_index = test[(test['date']!= price_time.loc[108,'date']) & (test['date']!= price_time.loc[160,'date'])].index

In [ ]:
r2_score(resale.iloc[test_index].resale_price.values,prediction)

In [ ]:
test =  resale[resale['year']>=2014]
predicted = pd.DataFrame({'y_pred':y_pred,
              'date':test['date']}).reset_index(drop=True)

prediction = [pred + mean_price[predicted.loc[i,'date']] for i,pred in enumerate(predicted['y_pred'])]